In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

# Configurar Selenium para ejecutar en modo headless
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

# Inicializar el driver de Selenium
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL base del sitio de noticias falsas
url = "https://www.newtral.es/zona-verificacion/fakes/"
driver.get(url)

# Lista para almacenar las noticias extraídas
news_data = []

In [17]:


def save_to_csv(filename="noticias_falsas.csv"):
    """
    Guarda las noticias recopiladas en un archivo CSV. Si el archivo no existe, lo crea; de lo contrario, agrega los datos.
    """
    df = pd.DataFrame(news_data)
    if news_saved == 0:
        df.to_csv(filename, index=False, encoding="utf-8")
    else:
        df.to_csv(filename, mode='a', index=False, header=False, encoding="utf-8")
    print(f"Se han guardado {len(news_data)} noticias en '{filename}'.")
    news_data.clear()

def extract_news(max_news, batch_size):
    """
    Extrae noticias de la página objetivo hasta 10000

    Args:
        max_news (int): Número máximo de noticias a extraer.
        batch_size (int): Cantidad de noticias tras las cuales guardar automáticamente.
    """
    global news_saved
    articles_extracted = 0
    while articles_extracted < max_news:
        try:
            # Encuentra las noticias en la página principal
            articles = driver.find_elements(By.CSS_SELECTOR, ".c-card__verification__quote a")
            for article in articles:
                if articles_extracted >= max_news:
                    break
                try:
                    time.sleep(3)  # Evitar bloqueos por acceso rápido
                    link = article.get_attribute("href")
                    driver.execute_script("window.open(arguments[0]);", link)
                    driver.switch_to.window(driver.window_handles[-1])

                    # Procesar el contenido de la noticia
                    soup = BeautifulSoup(driver.page_source, "html.parser")

                    # Extraer la fecha de publicación
                    date = soup.select_one("time.entry-date").text.strip() if soup.select_one("time.entry-date") else "Fecha no encontrada"

                    # Extraer el texto de verificación
                    verification_text = soup.select_one(".c-card__verification__quote-container").text.strip() if soup.select_one(".c-card__verification__quote-container") else "Verificación no encontrada"

                    # Almacenar los datos extraídos
                    news_data.append({
                        "Fecha": date,
                        "Verificación": verification_text,
                        "Enlace": link
                    })
                    articles_extracted += 1
                    news_saved += 1

                    # Guardar automáticamente tras alcanzar el batch_size
                    if len(news_data) >= batch_size:
                        save_to_csv()

                    # Cerrar la pestaña actual y volver a la principal
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    time.sleep(3)
                except Exception as e:
                    print(f"Error extrayendo noticia: {e}")
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    continue

            # Intentar cargar más noticias si no se alcanzó el límite
            if articles_extracted < max_news:
                load_more_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.ID, "scroll-load-more"))
                )
                load_more_button.click()
                time.sleep(5)
        except Exception as e:
            print("No se encontraron más noticias o hubo un error:", e)
            break

    # Guardar las noticias restantes
    if len(news_data) > 0:
        save_to_csv()

# Extraer noticias
extract_news(max_news=10000, batch_size=5)
print("Extracción completada.")

# Cerrar el navegador
driver.quit()
